# 1. Get dynamic pipeline parameters

In [ ]:
keyvaultlsname = 'Ls_KeyVault_01'
db_user_key='synapseSQLPoolAdminUsername'
db_pwd_key='synapseSQLPoolAdminPassword'

# 2. Get username password from keyvault

In [ ]:
from pyspark.sql import SparkSession
workspace_name=mssparkutils.env.getWorkspaceName()
print(workspace_name)

# 3. Get username password from keyvault

In [ ]:
# Make sure user has been created through create_db_user.sql before reading the data
sc = SparkSession.builder.getOrCreate()
token_library = sc._jvm.com.microsoft.azure.synapse.tokenlibrary.TokenLibrary
username = token_library.getSecretWithLS(keyvaultlsname, db_user_key)
password = token_library.getSecretWithLS(keyvaultlsname, db_pwd_key)


# 4. Get Data from external Table

In [ ]:
# Read Top 5 lines from the parking data
hostname = f"{workspace_name}-ondemand.sql.azuresynapse.net"
print(hostname)
port = 1433
database = "external_db" 
jdbcUrl = f"jdbc:sqlserver://{hostname}:{port};database={database}"
dbtable = "dbo.parking_bay_view"

#Parking Data
parking_data = spark.read \
    .format("com.microsoft.sqlserver.jdbc.spark") \
    .option("url", jdbcUrl) \
    .option("dbtable", dbtable) \
    .option("user", username) \
    .option("password", password) \
    .load()
print(parking_data.count())
parking_data.show(5)


In [ ]:
# Read Top 5 lines from the sensor data
dbtable = "sensor_view"
sensor_data = spark.read \
    .format("com.microsoft.sqlserver.jdbc.spark") \
    .option("url", jdbcUrl) \
    .option("dbtable", dbtable) \
    .option("user", username) \
    .option("password", password) \
    .load()
print(sensor_data.count())
sensor_data.show(5)